<a href="https://colab.research.google.com/github/alicelindel3/ibm5100/blob/main/1108/week4_home.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Homework3

下記の「#common」で始まるセルの中で指定されたリンク先にある購買データをベースに以下の処理を実行し、問いに答えて課題を提出してください。（データの前処理は、以下の記載の通りに行ってください。）
    
処理内容

1. `#common`のセルで事前に前処理されたデータ`target_online_retail_data_tb`を用います。
2. 購買者（`CustomerID`）の各合計購買金額（`TotalPrice`の合計）を算出し、さらに合計購買金額をベースに降順に並び替えをします。  
3. 合計購買金額の降順に並んだ購買者を$n$等分（$n$はデータ数を越えず、分位数に同一の値が存在しない自然数。購買者数が$n$で割り切れるとは限らず、この場合の処理は`Pandas.qcut`の処理に準じます。）にグループ分けします。  その際、合計購買金額が大きいグループから順に「グループ1」「グループ2」$\cdots$「グループn」とします。
4. それぞれのグループの金額合計値を算出してください（このアプローチをデシル分析といいます）。
5. それぞれのグループが売上の何割を占めるかを求めてください。その際、「グループ1」〜「グループn」の順に並べ、`Pandas.Series`形式で回答してください。


(なお、マーケティング戦略では、このように顧客を分けることをセグメンテーションといい、上位2割に絞ってアプローチを仕掛けることをターゲティングといいます。それぞれの戦略によりますが、優良顧客に的を絞った方が投資対効果が高いことが多いため、このようなアプローチを取ることがあります。これに関連して、上位2割の顧客が売上全体の8割を占めることを「パレートの法則」と言います。)

提出するときは、以下の点に注意してください。  
>- 以下の関数`homework`の`!!WRITE ME!!`に処理を書いてください。(**「`!!WRITE ME!!`」は消して、記入してください。**)
>- 実際の提出は記述された`homework`関数全てになり、**提出はOmnicampus内の宿題の欄から今週の課題を選択後、提出内容に関数を貼り付けてから[Pythonコード提出]を押してください。**
>- 前処理は以下のコードを使ってください。  
>- 返り値が`Pandas.Series`形式になるようにしてください。  
>- 解答は、`my_result=pd.Series([0.6,0.2,0.1,0.025,0.025,0.022,0.021,0.02,0.01,0]) `のような形になります。(売上割合はグループ順にして、合計は1になるようにします。)

In [1]:
# common
import pandas as pd
import numpy as np

# init part(データの読み込みと前処理)
file_url = "http://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx"
online_retail_data = pd.ExcelFile(file_url)
online_retail_data_table = online_retail_data.parse('Online Retail')

# 採点の都合上、文字列型に変換
online_retail_data_table['cancel_flg'] = online_retail_data_table.InvoiceNo.map(lambda x:str(x)[0])

# InvoiceNoの先頭が5であるものとIDがNullでないものが対象
target_online_retail_data_tb = online_retail_data_table[(online_retail_data_table.cancel_flg == '5') 
                                                        & (online_retail_data_table.CustomerID.notnull())]

target_online_retail_data_tb = target_online_retail_data_tb.assign(TotalPrice=target_online_retail_data_tb.Quantity * target_online_retail_data_tb.UnitPrice)

In [2]:
target_online_retail_data_tb

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,cancel_flg,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,5,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,5,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,5,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,5,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,5,20.34
...,...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France,5,10.20
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France,5,12.60
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France,5,16.60
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France,5,16.60


In [3]:
target_online_retail_data_tb.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,cancel_flg,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,5,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,5,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,5,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,5,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,5,20.34


In [4]:
target_online_retail_data_tb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 397924 entries, 0 to 541908
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    397924 non-null  object        
 1   StockCode    397924 non-null  object        
 2   Description  397924 non-null  object        
 3   Quantity     397924 non-null  int64         
 4   InvoiceDate  397924 non-null  datetime64[ns]
 5   UnitPrice    397924 non-null  float64       
 6   CustomerID   397924 non-null  float64       
 7   Country      397924 non-null  object        
 8   cancel_flg   397924 non-null  object        
 9   TotalPrice   397924 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(5)
memory usage: 33.4+ MB


In [5]:
df_descend = target_online_retail_data_tb[['CustomerID','TotalPrice']].groupby('CustomerID').sum().sort_values('TotalPrice', ascending=False)

df_descend

,TotalPrice
CustomerID,
14646.0,280206.02
18102.0,259657.30
17450.0,194550.79
16446.0,168472.50
14911.0,143825.06
...,...
17956.0,12.75
16454.0,6.90
14792.0,6.20


In [6]:
df_descend = target_online_retail_data_tb[['CustomerID','TotalPrice']].groupby('CustomerID').sum().sort_values('TotalPrice', ascending=False)
df_cut = pd.qcut(df_descend.TotalPrice,10)

df_cut

CustomerID
14646.0    (3646.164, 280206.02]
18102.0    (3646.164, 280206.02]
17450.0    (3646.164, 280206.02]
16446.0    (3646.164, 280206.02]
14911.0    (3646.164, 280206.02]
                   ...          
17956.0        (-0.001, 156.566]
16454.0        (-0.001, 156.566]
14792.0        (-0.001, 156.566]
16738.0        (-0.001, 156.566]
13256.0        (-0.001, 156.566]
Name: TotalPrice, Length: 4339, dtype: category
Categories (10, interval[float64, right]): [(-0.001, 156.566] < (156.566, 250.106] <
                                            (250.106, 351.658] < (351.658, 489.724] < ... <
                                            (941.942, 1353.74] < (1353.74, 2057.914] <
                                            (2057.914, 3646.164] < (3646.164, 280206.02]]

In [7]:
df_descend = target_online_retail_data_tb[['CustomerID','TotalPrice']].groupby('CustomerID').sum().sort_values('TotalPrice', ascending=False)
df_cut = pd.qcut(df_descend.TotalPrice,10)
cluster_customer_prices=df_descend.groupby(df_cut)['TotalPrice'].sum().sort_index(ascending=False)
#cluster_customer_prices=df_descend.groupby(df_cut)['TotalPrice'].sum().sort_values(ascending=False)

cluster_customer_prices

TotalPrice
(3646.164, 280206.02]    5469382.460
(2057.914, 3646.164]     1180055.001
(1353.74, 2057.914]       727103.640
(941.942, 1353.74]        493107.450
(674.45, 941.942]         343792.411
(489.724, 674.45]         252901.481
(351.658, 489.724]        179857.951
(250.106, 351.658]        132117.870
(156.566, 250.106]         87145.480
(-0.001, 156.566]          45944.160
Name: TotalPrice, dtype: float64

In [8]:
df_descend = target_online_retail_data_tb[['CustomerID','TotalPrice']].groupby('CustomerID').sum().sort_values('TotalPrice', ascending=False)
df_cut = pd.qcut(df_descend.TotalPrice,10)
cluster_customer_prices=df_descend.groupby(df_cut)['TotalPrice'].sum().sort_index(ascending=False)
All_price =target_online_retail_data_tb['TotalPrice'].sum()

All_price 

8911407.904

In [9]:
# working place. everything 
def homework(target_online_retail_data_tb, n):
    df_descend = target_online_retail_data_tb[['CustomerID','TotalPrice']].groupby('CustomerID').sum().sort_values('TotalPrice', ascending=False)
    df_cut = pd.qcut(df_descend.TotalPrice,n)
    cluster_customer_prices=df_descend.groupby(df_cut)['TotalPrice'].sum().sort_index(ascending=False)
    All_price =target_online_retail_data_tb['TotalPrice'].sum()
    my_result = cluster_customer_prices/All_price
    return my_result

In [10]:
homework(target_online_retail_data_tb, 10)

TotalPrice
(3646.164, 280206.02]    0.613751
(2057.914, 3646.164]     0.132421
(1353.74, 2057.914]      0.081592
(941.942, 1353.74]       0.055334
(674.45, 941.942]        0.038579
(489.724, 674.45]        0.028380
(351.658, 489.724]       0.020183
(250.106, 351.658]       0.014826
(156.566, 250.106]       0.009779
(-0.001, 156.566]        0.005156
Name: TotalPrice, dtype: float64

**謝辞**：以下のデータセットの利用に関して  
 https://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx

引用元：Dua, D. and Graff, C. (2019). UCI Machine Learning Repository [[http://archive.ics.uci.edu/ml](http://archive.ics.uci.edu/ml)]. Irvine, CA: University of California, School of Information and Computer Science.
 
Daqing Chen, Sai Liang Sain, and Kun Guo, Data mining for the online retail industry: A case study of RFM model-based customer segmentation using data mining, Journal of Database Marketing and Customer Strategy Management, Vol. 19, No. 3, pp. 197â€“208, 2012 (Published online before print: 27 August 2012. doi: 10.1057/dbm.2012.17).